# MNI Space Registration Example

This notebook performs a minimal, reproducible registration of a defaced T1 NIFTI into MNI space. The resulting file is intended as an example input for the `MONAILabel_Slicer_NIFTI_Demo` notebook.

For further examples on how to use FSL on JupyterLab for UKB-RAP, you can see this article on the community forum: https://community.ukbiobank.ac.uk/hc/en-gb/articles/30830097822237-JupyterLab-with-FSL-and-FreeSurfer


Estimated time: ~5 minutes. Recommended instance: `mem1_hdd1_v2_x16`. Kernel: ~Bash

When launching the JupyterLab instance, make sure to select the **IMAGE_PREPROCESSING** option in the Feature dropdown. 



In [ ]:
# This chunk downloads a single random defaced T1 NIFTI zip archive and extracts it to `subject01/`. 
# It also creates a text file `eid_path.txt` that records the participant ID and file path of subject01, so you can reference it later.

# Use field 20252 (T1 structural brain images - NIFTI)
field_id="20252"


# clear or create TXT file
output_txt="eid_path.txt"
> "$output_txt"

# Get 10 participants, shuffle, pick 1, and write eid + filepath (comma-separated) to TXT
dx find data --property field_id="$field_id" 2>/dev/null | \
awk -F'/' '
{
  start_index = index($0, "/Bulk/")
  end_index = index($0, " (file")
  filepath = substr($0, start_index, end_index - start_index)
  split($6, a, "_")
  eid = a[1]
  print eid "," filepath
}' | head -n 10 | shuf -n 1 >> "$output_txt"

# Make subject01 folder
mkdir -p subject01

# Download and unzip the file for the chosen participant
while IFS=',' read -r eid filepath; do
  echo "Downloading and extracting for participant: $eid"
  dx download --lightweight "${filepath}" -o "temp.zip"
  unzip temp.zip -d subject01/
  rm temp.zip
done < "$output_txt"

Next, we register the defaced, unprocessed T1 NIFTI file to MNI space using FSL `flirt`.

In [ ]:
flirt -in subject01/T1/T1_orig_defaced.nii.gz \
      -ref $FSLDIR/data/standard/MNI152_T1_1mm.nii.gz \
      -out subject01/T1/orig_MNI.nii.gz

Finally, we upload the registered image and the `eid_path.txt` back to your project. Adjust the `dx upload` path if you prefer a different destination.

In [ ]:
# Upload the MNI registered image and the eid_path file for later reference
dx upload subject01/T1/orig_MNI.nii.gz --path /subject01/T1/ --parents
dx upload eid_path.txt